In [1]:
import ee
import geemap
import geopandas as gpd
import numpy as np
from agroforestry.config import * 
from agroforestry.geeHelpers import *
from agroforestry.naipProcessing import *
from agroforestry.snicProcessing import *
from agroforestry.randomForest import *
from agroforestry.exportFunctions import *

In [2]:
# establish connection with ee account. might require some additional configuration based on local machine 
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()# i

In [3]:
# define the aoi
# aoiID = initGridID # something to itorate over for now is defined based on the input training dataset 
# or manually define it for where you want to apply the model too
# aoiID = "X12-601"
# # this becomes the AOI to used in the prepNAIP function. I'll need to edit it so that it converts the input data into a bbox 
# gridSelect = grid.loc[grid.Unique_ID == aoiID]


#Define bands to use
bandsToUse = vsurfWithCor

# select multiple grids level 1 
## want to pull thing from the csv rather than write it out. 
aoiID = ["X12-555","X12-556","X12-557","X12-600","X12-601","X12-602","X12-645","X12-646","X12-647"]
gridSelect =  grid.loc[grid.Unique_ID .isin(aoiID)].dissolve()
# gridSelect = grid.loc[grid.Unique_ID in aoiID]
# dissolve into a single object
gridSelect





,geometry,Unique_ID
0,"POLYGON ((-98.31460 42.43488, -98.31460 42.608...",X12-555


In [4]:
# convert to a gee object 
aoi1 = geemap.gdf_to_ee(gridSelect)
# create a sub grid for downloading 
# downloadGrids = geemap.fishnet(aoi1.geometry(), rows=6, cols=4, delta=0)
aoiID

['X12-555',
 'X12-556',
 'X12-557',
 'X12-600',
 'X12-601',
 'X12-602',
 'X12-645',
 'X12-646',
 'X12-647']

In [5]:
Map = geemap.Map(center=(42.3, -98), zoom=10)
Map.add_basemap('HYBRID')
Map.addLayer(aoi1, {'color': '000000ff',
                    'width': 2,
                    'lineType': 'solid'},
             'area of interest')
# Map.addLayer(downloadGrids, {'color': '000000ff',
#                     'width': 2,
#                     'lineType': 'solid'},
#              'area of subgrid')

Map

Map(center=[42.3, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [6]:
# import training dataset 
# trainingData = gpd.read_file(filename="data/processed/trainingdataset_withClasses.geojson")
trainingData = gpd.read_file(filename="data/processed/" + str(initGridID) +"/"+ "agroforestrySamplingData_"+str(year)+".geojson") # initGridID defined int he config file

# print(type(trainingData))
# select the training class of interest and drop unnecessary columns
trainingSubset =  trainingData[trainingData.sampleStrat == "original"] ## will want to drop this as we wont have multiple sampling categories 
# print(trainingSubset)
# convert to ee object
pointsEE = geemap.gdf_to_ee(gdf=trainingSubset)
# subset testing and training data 
training = pointsEE.filter(ee.Filter.gt('random', test_train_ratio))
testing = pointsEE.filter(ee.Filter.lte('random',test_train_ratio))
# traing the rf model 
# rfCluster = trainRFModel(bands=bandsToUse_Cluster, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
# rfPixel = trainRFModel(bands=bandsToUse_Pixel, inputFeature=training, nTrees=nTrees,setSeed=setSeed)
rfPixelTrim = trainRFModel(bands=bandsToUse,  inputFeature=training, nTrees=nTrees,setSeed=setSeed )
## run validation using the testing set 
# clusterValidation = testRFClassifier(classifier=rfCluster, testingData= testing)
# pixelValidation = testRFClassifier(classifier=rfPixel, testingData= testing)
pixelValidationTrim = testRFClassifier(classifier=rfPixelTrim, testingData= testing)


In [7]:
# cant print tuple with this function
geePrint(pixelValidationTrim)

0.9619771863117871


In [8]:

# define export aoi
# exportAOI = ee.Feature(downloadGrids.toList(50).get(4))
# geePrint(exportAOI)
# exportAOI.geometry()
year


2020

In [9]:
### 
# Generate model based on year define in config 

# generate NAIP layer 
naipEE = prepNAIP(aoi=aoi1, year=year,windowSize=windowSize)

# normal the naip data
# normalizedNAIP = normalize_by_maxes(img=naipEE, bandMaxes=bandMaxes)

# produce the SNIC object 
## filtering the image bands right away based on the single model output 
snicData = snicOutputs(naip = naipEE,
                       SNIC_SeedShape = SNIC_SeedShape, 
                       SNIC_SuperPixelSize = SNIC_SuperPixelSize, 
                       SNIC_Compactness = SNIC_Compactness, 
                       SNIC_Connectivity = SNIC_Connectivity,
                       # nativeScaleOfImage = nativeScaleOfImage, 
                       bandsToUse_Cluster = bandsToUse_Cluster).select(bandsToUse)
# apply the model and clip to aoi and reclass to unsigned 8bit image 
classifiedPixelsTrim = applyRFModel(imagery=snicData, bands=bandsToUse,classifier=rfPixelTrim).clip(aoi1).uint8()
# produce image for map 
demoImage = classifiedPixelsTrim #.clip(exportAOI)
# demoImage = classifiedPixelsTrim.clip(ee.Feature(downloadGrids.toList(50).get(10))).reproject(crs='EPSG:4326', scale=5)

geePrint(demoImage)


{'type': 'Image', 'bands': [{'id': 'classification', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [10]:
# print
# vsurfNoCor
# vsurfWithCor

In [11]:
# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#f5f7f710', '#10c9a1'],
}
# add features to the map
Map.addLayer(demoImage, vis_params, str(year) + ' model')
# Map.addLayer(demoImage2010, vis_params,'2010 model',False)
# Map.addLayer(demoImage2020, vis_params,'2020 model',False)

Map

Map(center=[42.3, -98], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [12]:
# export image to asset 
task = ee.batch.Export.image.toAsset(
  image = demoImage,
  description = 'fullAOI_level1surround2020',
  assetId = "projects/agroforestry2023/assets/testExport_10grids2020",
  region=aoi1.geometry(),
  scale=1,
  crs= demoImage.projection(),
  maxPixels = 1e13
)
task.start()

In [ ]:
# track the task 
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:

# image_new_class = demoImage.select('classification')
# geePrint(image_new_class)

# export_image = ee.batch.Export.image.toDrive(
#   image = image_new_class,
#   region = exportAOI.geometry(),
#   description = 'GTB_v',
#   folder = 'agroforestry',
#   scale = 30,
#   # crs = img_crs,
#   maxPixels = 1e13
#   )
  
#   #Check how many existing tasks are running and take a break of 5 mins if it's >10
#   # maximum_no_of_tasks(10, 5*60)
#   #Send next task.
# export_image.start()


# task =  ee.batch.Export.image.toDrive(
#     image=demoImage.select("classification"),
#     scale= 4,
#     description='testExport_0124',
#     folder='agroforestry',
#     # region= clipArea.geometry()
#     # maxPixels = 1e10
#     )
# task.start()
        

In [ ]:
# # test the export process 
# # A Landsat 8 surface reflectance image.
# image = ee.Image(
#     'LANDSAT/LC08/C02/T1_L2/LC08_044034_20210508'
# ).select(['SR_B.'])  # reflectance bands

# # A region of interest.
# region = ee.Geometry.BBox(-122.24, 37.13, -122.11, 37.20)

# # Set the export "scale" and "crs" parameters.
# task = ee.batch.Export.image.toDrive(
#     image=image,
#     description='image_export',
#     folder='agroforestry',
#     region=region,
#     scale=30,
#     crs='EPSG:5070'
# )
# task.start()


In [ ]:
geePrint(demoImage)